## 2022.1.5

In [3]:
import time
import random
import threading
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.lines as line
import numpy as np
%matplotlib qt5
from IPython import display
from queue import Queue
import numpy as np

import os
import subprocess

cmd1 = 'adb connect  10.10.25.2:5555'
subprocess.Popen(cmd1, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)

cur_cpu0=[] # 存放数据，用于绘制图像，数据类型可为列表

queue = Queue(maxsize=1000) #设置队列最大空间为1000
obsX = []
x0 = []

# 定义画布
fig = plt.figure()
ax = plt.subplot(111,ylim=(0,5))
line = line.Line2D([], [])  # 绘制直线

# 初始化图像
def plot_init():
    ax.add_line(line)
    return line, # 必须加逗号,否则会报错（TypeError: 'Line2D' object is not iterable）

# 更新图像(animation会不断调用此函数刷新图像，实现动态图的效果)
def plot_update(i):
#     global data  # data为全局变量
    global obsX, x0
#     print("obsX", obsX)
#     print("x0", x0)
    data_copy = x0.copy()  # 为避免线程不同步导致获取到的data在绘制图像时被更新，这里复制数据的副本，否则绘制图像的时候可能会出现x和y的数据维度不相等的情况
    x_data=np.arange(0,data_copy.shape[0],1)  # x轴根据y轴数据自动生成（可根据需要修改）
    ax.set_xlim(0,data_copy.shape[0])  # 横坐标范围（横坐标的范围和刻度可根据数据长度更新）
    print("data_copy", data_copy)
    print("x_data", x_data)
    ax.set_title("title",fontsize=8)  # 设置title
    line.set_xdata(x_data)  # 更新直线的数据
    line.set_ydata(data_copy)  # 更新直线的数据
	# 大标题（若有多个子图，可为其设置大标题）
    plt.suptitle('Suptitle',fontsize=8)
    # 重新渲染子图
    ax.figure.canvas.draw()  # 必须加入这一行代码，才能更新title和坐标!!!
    return line,  # 必须加逗号,否则会报错（TypeError: 'Line2D' object is not iterable）

# 绘制动态图
# ani = animation.FuncAnimation(fig,   # 画布
# 							  plot_update,  # 图像更新
#                               init_func=plot_init,  # 图像初始化
#                               frames=1,
#                               interval=30,  # 图像更新间隔
#                               blit=True)

def producer() :
#     lock.acquire()
    
    data = {}
    while True :
        
        cmdCpuCurFreq = 'adb shell cat /sys/devices/system/cpu/cpu' + "{}".format(0) + '/cpufreq/cpuinfo_cur_freq'
        curFreq_result = subprocess.Popen(cmdCpuCurFreq, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
        curFreq_output, curFreq_error = curFreq_result.communicate(timeout=60)
        cur_cpu0 = int(curFreq_output.decode()) / 1000000
        
        print("cur_cpu0", cur_cpu0)
        
        cur_time = time.time() 
#         cur_cpu0 = random.uniform(1.0, 2.5) #可以改成cpu的时间
        
        data = {
            "time" : cur_time,
            "cpu0" : cur_cpu0
        }
        queue.put(data)
#         print("producer")
#         time.sleep(0.1)
        
#         lock.release()

def customer() :
    
    global obsX, x0
    
    while True :
        if not queue.empty() :
            num = 100
            tmp_obsX, tmp_x0 = [], []
            while num > 0 :
                tmp = queue.get()

                _cur_time = list(tmp.values())[0]
                _cur_cpu0 = list(tmp.values())[1]

            
#             while num > 0 :
                tmp_obsX.append(_cur_time)
                tmp_x0.append(_cur_cpu0)
                num -= 1
            # 将列表转换为数组
            obsX = np.array(tmp_obsX)
            x0 = np.array(tmp_x0)
            
#             print("tmp_obsX", tmp_obsX)
#             print("tmp_x0", tmp_x0)
#             print("obsX", obsX)
#             print("x0", x0)

            num = 0
            tmp_obsX, tmp_x0 = [], []
#             print("customer")
# #             print("obsX", obsX)
#             print("x0", x0)
#             time.sleep(1)
        else :
            print('wait for seconds...')
            time.sleep(3)

# print("obsX", obsX)
# print("x0", x0)


p1_thread = threading.Thread(target=producer, args=())
c1_thread = threading.Thread(target=customer, args=())

p1_thread.daemon = True
c1_thread.daemon = True

p1_thread.start()
c1_thread.start()


plt.show()


data_copy [0.6144 0.6144 0.6144 0.6144 0.6144 0.6144 0.6144 0.6144 1.8048 1.0752
 1.3632 0.6144 0.6144 0.6144 0.6144 0.6144 0.6144 0.6144 0.6144 0.6144
 0.6144 0.6144 0.6144 0.6144 0.6144 0.6144 0.6144 0.6144 1.3632 0.6144
 0.6144 0.6144 0.6144 1.3632 0.6144 0.864  0.6144 0.6144 0.6144 1.0752
 0.6144 0.864  0.6144 1.0752 0.6144 0.6144 0.6144 0.6144 0.6144 0.6144
 0.6144 0.6144 0.864  0.6144 0.6144 0.6144 0.6144 0.6144 0.6144 0.6144
 0.6144 0.6144 0.6144 0.6144 0.864  0.6144 0.6144 0.6144 0.864  0.864
 1.0752 0.6144 0.6144 0.6144 0.6144 0.6144 0.6144 0.6144 0.6144 0.6144
 0.6144 0.864  0.864  0.6144 0.6144 1.0752 0.6144 0.6144 0.6144 0.6144
 0.6144 0.6144 0.864  1.8048 1.6512 0.6144 0.6144 0.6144 0.6144 0.6144]
x_data [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89

In [4]:
import os
import subprocess

```

/data/local/tmp/baoer/

#!/bin/bash
adb connect 10.10.25.2:5555
for((i=1;i<=50;i++));
do 
    current=`date "+%Y-%m-%d %H:%M:%S"`
    timeStamp=`date -d "$current" +%s` 
    currentTimeStamp=$((timeStamp*1000+10#`date "+%N"`/1000000)) #将current转换为时间戳，精确到毫秒
    
    echo $currentTimeStamp
    # echo $(expr $i \* 3 + 1);
    adb shell  cat /sys/devices/system/cpu/cpu0/cpufreq/cpuinfo_cur_freq
done
```

while True :
        
        cmdCpuCurFreq = 'adb shell cat /sys/devices/system/cpu/cpu' + "{}".format(0) + '/cpufreq/cpuinfo_cur_freq'
        curFreq_result = subprocess.Popen(cmdCpuCurFreq, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
        curFreq_output, curFreq_error = curFreq_result.communicate(timeout=60)
        cur_cpu0 = int(curFreq_output.decode()) / 1000000
        
        print("cur_cpu0", cur_cpu0)
#     print(cmdCpuCurFreq)

cur_cpu0 0.6144
cur_cpu0 0.6144
cur_cpu0 0.6144
cur_cpu0 1.8048
cur_cpu0 0.6144
cur_cpu0 0.6144
cur_cpu0 0.6144
cur_cpu0 1.5168
cur_cpu0 0.6144
cur_cpu0 0.6144
cur_cpu0 0.864
cur_cpu0 1.6512
cur_cpu0 0.6144
cur_cpu0 0.6144
cur_cpu0 0.6144
cur_cpu0 0.6144
cur_cpu0 1.0752
cur_cpu0 0.6144
cur_cpu0 1.0752
cur_cpu0 0.6144
cur_cpu0 1.3632
cur_cpu0 0.864
cur_cpu0 1.5168
cur_cpu0 0.6144
cur_cpu0 0.6144
cur_cpu0 0.6144
cur_cpu0 0.6144
cur_cpu0 0.6144
cur_cpu0 0.864
cur_cpu0 0.6144
cur_cpu0 1.3632
cur_cpu0 0.6144
cur_cpu0 1.0752
cur_cpu0 0.6144
cur_cpu0 0.6144
cur_cpu0 0.6144
cur_cpu0 0.6144
cur_cpu0 0.6144
cur_cpu0 0.6144
cur_cpu0 0.6144
cur_cpu0 0.864
cur_cpu0 0.6144
cur_cpu0 1.3632
cur_cpu0 1.8048
cur_cpu0 0.864
cur_cpu0 0.6144
cur_cpu0 0.6144
cur_cpu0 0.6144
cur_cpu0 1.0752
cur_cpu0 1.0752
cur_cpu0 0.6144
cur_cpu0 0.6144
cur_cpu0 0.6144
cur_cpu0 0.6144
cur_cpu0 0.6144
cur_cpu0 0.6144
cur_cpu0 1.5168
cur_cpu0 0.6144
cur_cpu0 0.6144
cur_cpu0 0.6144
cur_cpu0 1.3632
cur_cpu0 0.6144
cur_cpu0 1.36

KeyboardInterrupt: 

In [5]:
import time
import random
import threading
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.lines as line
import numpy as np
%matplotlib qt5
from IPython import display
from queue import Queue

CHUNK = 2048  # 初始数据量
data=np.random.normal(0,1,CHUNK)  # 存放数据，用于绘制图像，数据类型可为列表

# 定义画布
fig = plt.figure()
ax = plt.subplot(111,ylim=(0,5))
line = line.Line2D([], [])  # 绘制直线

# 初始化图像
def plot_init():
    ax.add_line(line)
    return line, # 必须加逗号,否则会报错（TypeError: 'Line2D' object is not iterable）

# 更新图像(animation会不断调用此函数刷新图像，实现动态图的效果)
def plot_update(i):
    global data  # data为全局变量
    data_copy = data.copy()  # 为避免线程不同步导致获取到的data在绘制图像时被更新，这里复制数据的副本，否则绘制图像的时候可能会出现x和y的数据维度不相等的情况
    x_data=np.arange(0,data_copy.shape[0],1)  # x轴根据y轴数据自动生成（可根据需要修改）
    
    print("x_data", x_data)
    print("data_copy", data_copy)
    
    ax.set_xlim(0,data_copy.shape[0])  # 横坐标范围（横坐标的范围和刻度可根据数据长度更新）
    ax.set_title("title",fontsize=8)  # 设置title
    line.set_xdata(x_data)  # 更新直线的数据
    line.set_ydata(data_copy)  # 更新直线的数据
	# 大标题（若有多个子图，可为其设置大标题）
    plt.suptitle('Suptitle',fontsize=8)
    # 重新渲染子图
    ax.figure.canvas.draw()  # 必须加入这一行代码，才能更新title和坐标!!!
    return line,  # 必须加逗号,否则会报错（TypeError: 'Line2D' object is not iterable）

# 绘制动态图
ani = animation.FuncAnimation(fig,   # 画布
							  plot_update,  # 图像更新
                              init_func=plot_init,  # 图像初始化
                              frames=1,
                              interval=30,  # 图像更新间隔
                              blit=True)

# 数据更新函数
def dataUpdate_thead():
    global data
    # 为了方便理解代码，这里生成正态分布的随机数据
    while True:  # 为了方便测试，让数据不停的更新
	    data=np.random.rand(CHUNK) #np.random.normal(0,1,CHUNK)

# 为数据更新函数单独创建一个线程，与图像绘制的线程并发执行
# ad_rdy_ev = threading.Event()
# ad_rdy_ev.set()  # 设置线程运行
t = threading.Thread(target=dataUpdate_thead, args=()) # 更新数据，参数说明：target是线程需要执行的函数，args是传递给函数的参数）
t.daemon = True
t.start()  # 线程执行

plt.show() # 显示图像(0,1,CHUNK)

# # 为数据更新函数单独创建一个线程，与图像绘制的线程并发执行
# ad_rdy_ev = threading.Event()
# ad_rdy_ev.set()  # 设置线程运行
# t = threading.Thread(target=dataUpdate_thead, args=()) # 更新数据，参数说明：target是线程需要执行的函数，args是传递给函数的参数）
# t.daemon = True
# t.start()  # 线程执行

# plt.show() # 显示图像


x_data [   0    1    2 ... 2045 2046 2047]
data_copy [0.15799986 0.57296723 0.52686366 ... 0.4591003  0.50212822 0.36756789]
x_data [   0    1    2 ... 2045 2046 2047]
data_copy [0.49492808 0.99148115 0.84294556 ... 0.64890077 0.43754022 0.10640937]
x_data [   0    1    2 ... 2045 2046 2047]
data_copy [0.55805173 0.7315256  0.6698239  ... 0.65852353 0.20054217 0.02079844]
x_data [   0    1    2 ... 2045 2046 2047]
data_copy [0.31953139 0.03799951 0.41398144 ... 0.882743   0.32583195 0.4615704 ]
x_data [   0    1    2 ... 2045 2046 2047]
data_copy [0.08094246 0.28902884 0.50939262 ... 0.45883163 0.66961991 0.17154313]
x_data [   0    1    2 ... 2045 2046 2047]
data_copy [0.3599434  0.52361184 0.16954187 ... 0.03478086 0.88739309 0.82379266]
x_data [   0    1    2 ... 2045 2046 2047]
data_copy [0.69865738 0.51455988 0.56958302 ... 0.51949749 0.74887955 0.59724573]
x_data [   0    1    2 ... 2045 2046 2047]
data_copy [0.46936984 0.31647476 0.33331643 ... 0.97712722 0.91525845 0.15000595]


In [6]:
import numpy as np
import random
list1 = []

for i in range(10) :
    list1.append(random.random())

test = np.array(list)    
    

cur_time = random.random()
cur_cpu0 = random.uniform(1.0, 2.5) #可以改成cpu的时间

data = {
    "time" : cur_time,
    "cpu0" : cur_cpu0
}

print(list(data.values())[0])
# data.values()

0.03642857109631181


In [4]:
import os
import subprocess

cmd1 = 'adb connect  10.10.25.2:5555'
subprocess.Popen(cmd1, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)

cmdCpuCurFreq = 'adb shell cat /sys/devices/system/cpu/cpu' + "{}".format(0) + '/cpufreq/cpuinfo_cur_freq'
curFreq_result = subprocess.Popen(cmdCpuCurFreq, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
curFreq_output, curFreq_error = curFreq_result.communicate(timeout=60)
cur_cpu0 = int(curFreq_output.decode()) / 1000000

print(curFreq_output.decode())
print(cur_cpu0)

614400

0.6144
